In [1]:
# the following script contains portions of code used by Digital Earth Australia in their tutorials:
# https://docs.dea.ga.gov.au/notebooks/Scientific_workflows/TSmask/TSmask.html, 
# under Apache License, Version 2.0 (https://www.apache.org/licenses/LICENSE-2.0)
# DEA data is under Creative Commons by Attribution 4.0 license 
# (https://creativecommons.org/licenses/by/4.0/)

In [2]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import datacube
import xarray as xr
from datacube.utils.masking import make_mask
from datacube.drivers.netcdf import write_dataset_to_netcdf
import numpy as np
from numpy import inf
import pandas as pd
from datetime import timedelta,datetime
import warnings
warnings.simplefilter("ignore") 
#sys.path

## Download and load the Random Forest model.

In [3]:
!pip3 install pickle5

In [4]:
import pickle5 as pickle
with open('rf_fmc.pickle', 'rb') as handle:
    rf = pickle.load(handle)
    print(rf)

RandomForestRegressor(criterion='mse', max_depth=25, max_features='auto',
                      n_estimators=25, n_jobs=8)


## Load DEA data for region of interest

In [5]:
dc = datacube.Datacube(app='fmc')

In [6]:
# Set some configurations for displaying tables nicely
#pd.set_option("display.max_colwidth", 200)
#pd.set_option("display.max_rows", None)

#products = dc.list_products()
#products

In [7]:
#check variables names
product = "ga_s2am_ard_3"
measurements = dc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata,aliases,flags_definition
measurement,,,,,,
nbart_coastal_aerosol,nbart_coastal_aerosol,int16,1,-999,"[nbart_band01, coastal_aerosol]",NaN
nbart_blue,nbart_blue,int16,1,-999,"[nbart_band02, blue]",NaN
nbart_green,nbart_green,int16,1,-999,"[nbart_band03, green]",NaN
nbart_red,nbart_red,int16,1,-999,"[nbart_band04, red]",NaN
nbart_red_edge_1,nbart_red_edge_1,int16,1,-999,"[nbart_band05, red_edge_1]",NaN
nbart_red_edge_2,nbart_red_edge_2,int16,1,-999,"[nbart_band06, red_edge_2]",NaN
nbart_red_edge_3,nbart_red_edge_3,int16,1,-999,"[nbart_band07, red_edge_3]",NaN
nbart_nir_1,nbart_nir_1,int16,1,-999,"[nbart_band08, nir_1, nbart_common_nir]",NaN
nbart_nir_2,nbart_nir_2,int16,1,-999,"[nbart_band8a, nir_2]",NaN


In [8]:
#check variables names
product = "ga_s2bm_ard_3"
measurements = dc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata,aliases,flags_definition
measurement,,,,,,
nbart_coastal_aerosol,nbart_coastal_aerosol,int16,1,-999,"[nbart_band01, coastal_aerosol]",NaN
nbart_blue,nbart_blue,int16,1,-999,"[nbart_band02, blue]",NaN
nbart_green,nbart_green,int16,1,-999,"[nbart_band03, green]",NaN
nbart_red,nbart_red,int16,1,-999,"[nbart_band04, red]",NaN
nbart_red_edge_1,nbart_red_edge_1,int16,1,-999,"[nbart_band05, red_edge_1]",NaN
nbart_red_edge_2,nbart_red_edge_2,int16,1,-999,"[nbart_band06, red_edge_2]",NaN
nbart_red_edge_3,nbart_red_edge_3,int16,1,-999,"[nbart_band07, red_edge_3]",NaN
nbart_nir_1,nbart_nir_1,int16,1,-999,"[nbart_band08, nir_1, nbart_common_nir]",NaN
nbart_nir_2,nbart_nir_2,int16,1,-999,"[nbart_band8a, nir_2]",NaN


In [9]:
ds = pd.read_csv('ignition2018_lfmc.csv')
df = ds.copy()
df.head()

,Unnamed: 0,syd_time,utc_time,syd_day,uts_day,event_type,latitude,longitude,amperage,index_Igni,...,LFMCigni_modis_date_closest,LFMCigni_modis_date_2ndclosest,LFMCigni_modis_lagDays_closest,LFMCigni_modis_lagDays_2ndclosest,LFMClight_s2_closest,LFMClight_s2_date_closest,LFMClight_s2_lagDays,LFMCigni_s2_closest,LFMCigni_s2_date_closest,LFMCigni_s2_lagDays
0,19,3/01/2018 14:12,3/01/2018 03:12,3/01/2018,3/01/2018,GS,-30.47615,151.67750,-16.2,1116,...,1/01/2018,31/12/2017,2,3,NaN,NaN,NaN,NaN,NaN,NaN
1,296,25/01/2018 19:07,25/01/2018 08:07,25/01/2018,25/01/2018,GS,-29.92114,151.28798,-6.4,1125,...,21/01/2018,17/01/2018,4,8,NaN,NaN,NaN,NaN,NaN,NaN
2,588,25/01/2018 21:08,25/01/2018 10:08,25/01/2018,25/01/2018,GS,-29.98587,151.25208,-3.9,1126,...,21/01/2018,17/01/2018,4,8,NaN,NaN,NaN,NaN,NaN,NaN
3,767,26/01/2018 18:33,26/01/2018 07:33,26/01/2018,26/01/2018,GS,-29.96574,151.17443,-18.1,1127,...,25/01/2018,21/01/2018,1,5,NaN,NaN,NaN,NaN,NaN,NaN
4,840,17/02/2018 14:09,17/02/2018 03:09,17/02/2018,17/02/2018,GS,-30.47077,151.71985,-5.3,1141,...,14/02/2018,10/02/2018,3,7,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def convert_to_datetime(date_string):
    #using negative indeces becasue sometimes day is one digit, e.g. 3/01/2018
    y = int(date_string[-4:])
    m = int(date_string[-7:-5])
    d = int(date_string[:-8])

    return datetime(y,m,d)




In [ ]:
#lightning

for i in df.index:
    print(i+1,'/',len(df.index))
    
    lat = df['latitude'][i]   ###########################################################################################################################################################
    lon = df['longitude'][i]   ##########################################################################################################################################################
    date = convert_to_datetime(df['syd_day'][i])    #####################################################################################################################################
    #set range of search for sentinel2 data (in this case, 30 days before event, event date excluded, just to be sure to get at least one image per satellite so that the program does not stop for a missing time dimension error)
    date_startsearch = (date-timedelta(days=31)).strftime('%Y-%m-%d') 
    date_endsearch = (date-timedelta(days=1)).strftime('%Y-%m-%d') 
    
    query = {
        'y': lat,
        'x': lon,
        'crs': 'EPSG:4326',
        'output_crs': 'EPSG:4326',
        'resolution': (-0.0002, 0.0002),  #some bands are 10m resolution, but others are 20m
        'time': (date_startsearch, date_endsearch),
        'measurements':["oa_fmask",
                         "nbart_blue","nbart_green","nbart_red",
                         "nbart_red_edge_1","nbart_red_edge_2","nbart_red_edge_3",
                         "nbart_nir_1","nbart_nir_2",
                         "nbart_swir_2","nbart_swir_3"], }
    
    s2a_ds = dc.load(product='ga_s2am_ard_3', group_by='solar_day', **query)
    s2b_ds = dc.load(product='ga_s2bm_ard_3', group_by='solar_day', **query)
    ds_nomask = xr.concat([s2a_ds, s2b_ds], dim='time').sortby('time')
    
    df.loc[i,'LFMClight_s2_date_closest'] = pd.to_datetime(ds_nomask.time.data[-1]).replace(hour=0,minute=0,second=0,microsecond=0).strftime('%d/%m/%Y') ####################################
    df.loc[i,'LFMClight_s2_lagDays'] = (date - pd.to_datetime(ds_nomask.time.data[-1]).replace(hour=0,minute=0,second=0,microsecond=0)).days ##################################################

    ds_nomask_latest = ds_nomask.isel(time=-1) #latest available

    ds_nomask_latest['ndvi']=((ds_nomask_latest.nbart_nir_1-ds_nomask_latest.nbart_red)/(ds_nomask_latest.nbart_nir_1+ds_nomask_latest.nbart_red))
    ds_nomask_latest['ndii']=((ds_nomask_latest.nbart_nir_1-ds_nomask_latest.nbart_swir_2)/(ds_nomask_latest.nbart_nir_1+ds_nomask_latest.nbart_swir_2))

    # Stack and reshape dataset to be compatible with the RF input
    refl = ds_nomask_latest[['ndvi','ndii','nbart_red','nbart_green','nbart_blue','nbart_nir_1','nbart_nir_2','nbart_swir_2','nbart_swir_3']].to_array().values

    # Convert infinite values to nans
    refl = np.nan_to_num(refl, copy=False, nan= np.nan, posinf=np.nan, neginf=np.nan)

    # Check and mask values not accepted by RF model
    nan_mask = np.isnan(refl)

    if np.sum(np.where(nan_mask,1,0))==0 and ds_nomask_latest.oa_fmask.data==1: #i.e., no refl band value is missing and no cloud/snow/shadow/water
        refl_rf = refl.reshape((9,-1)).swapaxes(0,1)
        rf_lfmc = rf.predict(refl_rf)
        lfmc_array = rf_lfmc.reshape(refl.shape[1:])
        lfmc_value = lfmc_array[0,0] #basically it's a 2d array with only one value

        df.loc[i,'LFMClight_s2_closest'] = lfmc_value  #####################################################################################################################################
    
    if i in range(50,450,50) or i==len(df.index)-1:
        df.to_csv('ignition2018_lfmc_modis_s2_templight{}.csv'.format(i),index=False)



In [ ]:
# ignition

for i in df.index:
    print(i+1,'/',len(df.index))
    
    lat = df['Y'][i]    ################################################################################################################################################################
    lon = df['X'][i]    ################################################################################################################################################################
    date = convert_to_datetime(df['StartDt'][i])  #####################################################################################################################################
    #set range of search for sentinel2 data (in this case, 30 days before event, event date excluded, just to be sure to get at least one image per satellite so that the program does not stop for a missing time dimension error)
    date_startsearch = (date-timedelta(days=31)).strftime('%Y-%m-%d') 
    date_endsearch = (date-timedelta(days=1)).strftime('%Y-%m-%d') 
    
    query = {
        'y': lat,
        'x': lon,
        'crs': 'EPSG:4326',
        'output_crs': 'EPSG:4326',
        'resolution': (-0.0002, 0.0002),  #some bands are 10m resolution, but others are 20m
        'time': (date_startsearch, date_endsearch),
        'measurements':["oa_fmask",
                         "nbart_blue","nbart_green","nbart_red",
                         "nbart_red_edge_1","nbart_red_edge_2","nbart_red_edge_3",
                         "nbart_nir_1","nbart_nir_2",
                         "nbart_swir_2","nbart_swir_3"], }
    
    s2a_ds = dc.load(product='ga_s2am_ard_3', group_by='solar_day', **query)
    s2b_ds = dc.load(product='ga_s2bm_ard_3', group_by='solar_day', **query)
    ds_nomask = xr.concat([s2a_ds, s2b_ds], dim='time').sortby('time')
    
    df.loc[i,'LFMCigni_s2_date_closest'] = pd.to_datetime(ds_nomask.time.data[-1]).replace(hour=0,minute=0,second=0,microsecond=0).strftime('%d/%m/%Y') ######################################
    df.loc[i,'LFMCigni_s2_lagDays'] = (date - pd.to_datetime(ds_nomask.time.data[-1]).replace(hour=0,minute=0,second=0,microsecond=0)).days ####################################################

    ds_nomask_latest = ds_nomask.isel(time=-1) #latest available

    ds_nomask_latest['ndvi']=((ds_nomask_latest.nbart_nir_1-ds_nomask_latest.nbart_red)/(ds_nomask_latest.nbart_nir_1+ds_nomask_latest.nbart_red))
    ds_nomask_latest['ndii']=((ds_nomask_latest.nbart_nir_1-ds_nomask_latest.nbart_swir_2)/(ds_nomask_latest.nbart_nir_1+ds_nomask_latest.nbart_swir_2))

    # Stack and reshape dataset to be compatible with the RF input
    refl = ds_nomask_latest[['ndvi','ndii','nbart_red','nbart_green','nbart_blue','nbart_nir_1','nbart_nir_2','nbart_swir_2','nbart_swir_3']].to_array().values

    # Convert infinite values to nans
    refl = np.nan_to_num(refl, copy=False, nan= np.nan, posinf=np.nan, neginf=np.nan)

    # Check and mask values not accepted by RF model
    nan_mask = np.isnan(refl)

    if np.sum(np.where(nan_mask,1,0))==0 and ds_nomask_latest.oa_fmask.data==1: #i.e., no refl band value is missing and no cloud/snow/shadow/water
        refl_rf = refl.reshape((9,-1)).swapaxes(0,1)
        rf_lfmc = rf.predict(refl_rf)
        lfmc_array = rf_lfmc.reshape(refl.shape[1:])
        lfmc_value = lfmc_array[0,0] #basically it's a 2d array with only one value

        df.loc[i,'LFMCigni_s2_closest'] = lfmc_value  #####################################################################################################################################
        
    if i in range(50,450,50) or i==len(df.index)-1:
        df.to_csv('ignition2018_lfmc_modis_s2_tempigni{}.csv'.format(i),index=False)
    

In [ ]:
df.to_csv('ignition2018_lfmc_modis_s2.csv',index=False)